## HomeIQ API test checks

#### NZTA EV API Data 

In [ ]:
import requests
import pandas as pd

# ───────────────────────────────────────────────────────────────
# CONFIG – July 2025 snapshot endpoint
# ───────────────────────────────────────────────────────────────
BASE_URL  = "https://services.arcgis.com/CXBb7LAjgIIdcsPt/" \
            "arcgis/rest/services/MVR_Jul2025/FeatureServer/0"
QUERY_URL = f"{BASE_URL}/query"

# ───────────────────────────────────────────────────────────────
# Core: paged fetch with guard-rails
# ───────────────────────────────────────────────────────────────
def fetch_mvr_slice(where="1=1", fields="*", geom=False, page_size=2000):
    """
    Streams slices of the NZTA Motor Vehicle Register.

    Parameters
    ----------
    where     : str  ArcGIS SQL filter (e.g. "MOTIVE_POWER = ELECTRIC, PLUGIN HYBRID etc")
    fields    : str  Comma-separated list or "*" for all attributes
    geom      : bool Include WGS-84 geometry (lat/lon) if True
    page_size : int  ≤ layer's maxRecordCount (usually 2000)

    Returns
    -------
    pandas.DataFrame
    """
    offset, rows = 0, []
    base_params = {
        "where": where,
        "outFields": fields,
        "returnGeometry": str(geom).lower(),
        "resultRecordCount": page_size,
        "f": "json"
    }
    if geom:
        base_params["outSR"] = 4326

    while True:
        resp = requests.get(QUERY_URL, params={**base_params, "resultOffset": offset}, timeout=60)
        resp.raise_for_status()
        js = resp.json()

        # ── error handling ──────────────────────────────────────
        if "error" in js:
            e = js["error"]
            raise RuntimeError(f"ArcGIS error {e['code']}: {e['message']}")
        feats = js.get("features")
        if feats is None:
            raise RuntimeError(f"Unexpected response keys: {list(js.keys())}")
        # ─────────────────────────────────────────────────────────

        rows.extend(f["attributes"] for f in feats)
        if not js.get("exceededTransferLimit"):
            break
        offset += page_size

    return pd.DataFrame(rows)


# ───────────────────────────────────────────────────────────────
# Electric & Hybrid Vehicle Retrieval Function
# ───────────────────────────────────────────────────────────────
def fetch_electric_vehicles(min_year=2020, fields="*"):
    """
    Fetches electric and hybrid vehicles from the NZTA Motor Vehicle Register.
    
    Parameters
    ----------
    min_year : int  Minimum first NZ registration year (default: 2020)
    fields   : str  Comma-separated list or "*" for all attributes
    
    Returns
    -------
    pandas.DataFrame
        DataFrame containing electric and hybrid vehicle records
    """
    # Build comprehensive electric/hybrid vehicle filter
    ev_where = (
        "(MOTIVE_POWER LIKE '%PLUGIN PETROL HYBRID%' OR "
        "MOTIVE_POWER LIKE '%ELECTRIC%' OR "
        "MOTIVE_POWER LIKE '%PLUGIN DIESEL HYBRID%' OR "
        "MOTIVE_POWER LIKE '%PLUG IN FUEL CELL HYDROGEN HYBRID%' OR "
        "MOTIVE_POWER LIKE '%PLUG IN FUEL CELL OTHER HYBRID%') AND "
        f"FIRST_NZ_REGISTRATION_YEAR >= {min_year}"
    )
    
    return fetch_mvr_slice(where=ev_where, fields=fields)


# ───────────────────────────────────────────────────────────────
# DEMO: Electric vehicles first registered in 2020 or later
# ───────────────────────────────────────────────────────────────
df_evs = fetch_electric_vehicles(min_year=2020, fields="*")
print("EVs & PHEVs ≥ 2020:", df_evs.shape)
print("\nAll available features:")
print(df_evs.columns.tolist())
print("\nFirst 10 records:")
df_evs.head(10)


EVs & PHEVs ≥ 2020: (117519, 39)

All available features:
['OBJECTID', 'ALTERNATIVE_MOTIVE_POWER', 'BASIC_COLOUR', 'BODY_TYPE', 'CC_RATING', 'CHASSIS7', 'CLASS', 'ENGINE_NUMBER', 'FIRST_NZ_REGISTRATION_YEAR', 'FIRST_NZ_REGISTRATION_MONTH', 'GROSS_VEHICLE_MASS', 'HEIGHT', 'IMPORT_STATUS', 'INDUSTRY_CLASS', 'INDUSTRY_MODEL_CODE', 'MAKE', 'MODEL', 'MOTIVE_POWER', 'MVMA_MODEL_CODE', 'NUMBER_OF_AXLES', 'NUMBER_OF_SEATS', 'NZ_ASSEMBLED', 'ORIGINAL_COUNTRY', 'POWER_RATING', 'PREVIOUS_COUNTRY', 'ROAD_TRANSPORT_CODE', 'SUBMODEL', 'TLA', 'TRANSMISSION_TYPE', 'VDAM_WEIGHT', 'VEHICLE_TYPE', 'VEHICLE_USAGE', 'VEHICLE_YEAR', 'VIN11', 'WIDTH', 'SYNTHETIC_GREENHOUSE_GAS', 'FC_COMBINED', 'FC_URBAN', 'FC_EXTRA_URBAN']

First 10 records:


,OBJECTID,ALTERNATIVE_MOTIVE_POWER,BASIC_COLOUR,BODY_TYPE,CC_RATING,CHASSIS7,CLASS,ENGINE_NUMBER,FIRST_NZ_REGISTRATION_YEAR,FIRST_NZ_REGISTRATION_MONTH,...,VDAM_WEIGHT,VEHICLE_TYPE,VEHICLE_USAGE,VEHICLE_YEAR,VIN11,WIDTH,SYNTHETIC_GREENHOUSE_GAS,FC_COMBINED,FC_URBAN,FC_EXTRA_URBAN
0,1531074,None,BLACK,STATION WAGON,1969,None,MA,3012729,2020,2,...,0,PASSENGER CAR/VAN,PRIVATE PASSENGER,2020,YV1LFBAADK1,0,HFC-134A (R134A),2.1,2.1,2.1
1,1584021,None,WHITE,SALOON,1,None,MA,T19F1532857,2020,1,...,0,PASSENGER CAR/VAN,PRIVATE PASSENGER,2020,5YJSB7E25KF,0,HFC-134A (R134A),None,None,None
2,1584059,None,WHITE,MOTORCYCLE,1,None,LA,UFD12A201909034,2020,1,...,0,MOPED,PRIVATE PASSENGER,2020,L5XU1AAB8J6,0,None,None,None,None
3,1782042,None,SILVER,HATCHBACK,1797,ZVW35-3,MA,2ZR-5444171,2021,7,...,0,PASSENGER CAR/VAN,PRIVATE PASSENGER,2012,7AT0H637X19,0,HFC-134A (R134A),None,None,None
4,1849232,None,WHITE,STATION WAGON,0,None,MA,EM16LB5253BJ,2021,1,...,0,PASSENGER CAR/VAN,PRIVATE PASSENGER,2021,KMHK281GULU,0,HFO-1234YF (R1234YF),None,None,None
5,1849239,None,RED,SALOON,0,None,MA,TG121005000BVL,2021,3,...,0,PASSENGER CAR/VAN,PRIVATE PASSENGER,2021,LRW3F7FA5MC,0,HFC-134A (R134A),None,None,None
6,1849266,None,WHITE,SALOON,0,None,MA,TG121086000HS1,2021,6,...,0,PASSENGER CAR/VAN,PRIVATE PASSENGER,2021,LRW3F7EB6MC,0,HFC-134A (R134A),None,None,None
7,1849268,None,BLUE,SALOON,0,None,MA,TG1210960001HZ,2021,6,...,0,PASSENGER CAR/VAN,PRIVATE PASSENGER,2021,LRW3F7EB7MC,0,HFC-134A (R134A),None,None,None
8,1849292,None,SILVER,SALOON,0,None,MA,TG3211670012RV,2021,9,...,0,PASSENGER CAR/VAN,PRIVATE PASSENGER,2021,LRW3F7EL3MC,0,HFC-134A (R134A),None,None,None
9,1849300,None,WHITE,SALOON,0,None,MA,TG121154002MVT,2021,9,...,0,PASSENGER CAR/VAN,PRIVATE PASSENGER,2021,LRW3F7FA3MC,0,HFC-134A (R134A),None,None,None


### LINZ NZ Addresses API

In [ ]:
## LINZ NZ Addresses API

In [ ]:
import requests
import geopandas as gpd
import json

def fetch_nz_addresses(api_key, max_features, bbox=None, cql_filter=None):
    """
    Fetches data from the LINZ NZ Addresses WFS API.
    
    Parameters:
    - api_key: LINZ Data Service API key (str)
    - max_features: Maximum number of features to return (int, optional)
    - bbox: Bounding box tuple (minx, miny, maxx, maxy) for spatial filter (optional)
    - cql_filter: CQL filter string for attribute filtering (optional, e.g., "address LIKE '%Auckland%'")
    
    Returns:
    - GeoDataFrame with the fetched address data
    """
    url = f"https://data.linz.govt.nz/services;key={api_key}/wfs"
    params = {
        "service": "WFS",
        "version": "2.0.0",
        "request": "GetFeature",
        "typeNames": "layer-105689",  # NZ Addresses layer ID
        "outputFormat": "json",
        "count": max_features  # Limit the number of features
    }
    
    # Add optional spatial filter (BBOX)
    if bbox:
        params["bbox"] = ",".join(map(str, bbox))
     
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an error if the request fails
    
    # Load the GeoJSON response into a GeoDataFrame
    features = response.json()
    gdf = gpd.GeoDataFrame.from_features(features)
    
    return gdf

# Set API key and fetch all NZ addresses
api_key = "4e3e6c63cdc541a295a8cb23775be6d4"  # This is my API key. You can replace with your own API key
data = fetch_nz_addresses(api_key, max_features=None) # set max_features to None to grab all addresses or to 10 to grab 10 addresses etc. 
print(data.head())  # Display the first few rows of the data


                      geometry  address_id source_dataset  change_id  \
0  POINT (175.05539 -39.93448)     2394435           AIMS    8351181   
1  POINT (175.05539 -39.93448)     2394436           AIMS    8351193   
2  POINT (175.05539 -39.93448)     2394437           AIMS    8351194   
3  POINT (175.05539 -39.93448)     2394438           AIMS    8351195   
4  POINT (175.05539 -39.93448)     2394439           AIMS    8351196   

  full_address_number   full_road_name  \
0         Floor 1 1/2  Victoria Avenue   
1         Floor 1 2/2  Victoria Avenue   
2         Floor 1 3/2  Victoria Avenue   
3         Floor 1 4/2  Victoria Avenue   
4         Floor 1 5/2  Victoria Avenue   

                             full_address territorial_authority unit_type  \
0  Floor 1 1/2 Victoria Avenue, Whanganui    Whanganui District      Unit   
1  Floor 1 2/2 Victoria Avenue, Whanganui    Whanganui District      Unit   
2  Floor 1 3/2 Victoria Avenue, Whanganui    Whanganui District      Unit   
3  Flo

In [17]:
data.shape

(1000, 35)